In [48]:
import os
import json
import csv
import copy
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [49]:
import sys
# sys.path.append('../')
# from tools.preprocess import Preprocessor

In [50]:
def json2data(path):
    cols = ["text", "label", "subLabel"]
    df = pd.DataFrame(index=[], columns=cols)
    files = os.listdir(path)
    for cop in files:
        if "." not in cop:
            continue
        with open(path+cop, "r") as f:
            json_data = json.load(f)
            mode = cop.split(".")[0]
            for data in json_data[mode]:
                text = data["data"]
                label = data["label"][0]
                subLabel = ""
                df = df.append(pd.DataFrame([text, label, subLabel], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

label_list = "YN WH please plain".split()
label_dict = dict( zip(label_list, range(len(label_list))) )

def extract_X_y(df):
    X = []
    y = []
    for te, la in zip(df.text, df.label):
        X.append(te)
        y.append(label_dict[la])
    return X, y
    

In [51]:
path = "../corpus/data_augment.csv"
data = []
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

In [52]:
def make_classes_dict(data):
    classes_dict = dict()
    for row in data:
        c = row[1]
        if c not in classes_dict:
            classes_dict[c] = len(classes_dict)
    return classes_dict

def class_filter(data, remain_classes:list):
    new_data = []
    for row in data:
        c = row[1]
        if c in remain_classes:
            new_data.append(row)
    return new_data

In [53]:
classes_dict_ = make_classes_dict(data)

In [54]:
remain_classes = "how what when where who why YN plain".split()
data_n = class_filter(data, remain_classes)
classes_dict = dict(zip(remain_classes, list(range(len(remain_classes)))))

In [55]:
import random


def extract_X_y_limit(data, classes_dict:dict, limit=600):
    data_ = random.sample(data, len(data))
    X = []
    y = []
    each_len = dict(zip(classes_dict.keys(), [0]*len(classes_dict)))
    for d in data_:
        if each_len[d[1]] <= limit:
            X.append(d[0])
            y.append(classes_dict[d[1]])
            each_len[d[1]] += 1
    return X, y

In [56]:
X, y = extract_X_y_limit(data_n, classes_dict)

In [57]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [59]:
import spacy
nlp = spacy.load('ja_ginza')
import torch.nn.utils.rnn as rnn
def make_Xseq(X):
    Xseq = [ torch.tensor( [token.vector  for token in nlp(x)] ) for x in X]
    Xseq = rnn.pad_sequence(Xseq, batch_first=True)
    # print(len(Xseq))
    # for xs in Xseq:
    #     print(xs.shape)
    return Xseq


In [60]:
X_train = make_Xseq(X_train_str)

In [61]:
class Datasets(torch.utils.data.Dataset):
    def __init__(self, X_data, y_data):
        # self.transform = transform

        self.X_data = X_data
        self.y_data = y_data

        self.datanum = len(X_data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_X = self.X_data[idx]
        out_y = self.y_data[idx]

        return out_X, out_y

In [63]:
X_train = X_train[:-5]
y_train = y_train[:-5]
leng = len(y_train)
print(leng)
for i, v in enumerate(y_train):
    if leng %(i+1) == 0:
        print(i+1, end=", ")

3360
1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 15, 16, 20, 21, 24, 28, 30, 32, 35, 40, 42, 48, 56, 60, 70, 80, 84, 96, 105, 112, 120, 140, 160, 168, 210, 224, 240, 280, 336, 420, 480, 560, 672, 840, 1120, 1680, 3360, 

In [71]:
BATCH_SIZE = 80
epoch_ = 800
trainset = Datasets(X_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

In [72]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, tagset_size, batch_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。    
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        # LSTMの隠れ層。これ１つでOK。超便利。
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, batch_first=True, bidirectional=True)
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        # self.softmax = 
    
    def forward(self, x):
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        batch_size, seq_len = x.shape[0], x.shape[1]
        _, hidden_layer = self.lstm(x)
        # print(hidden_layer)
        y = self.hidden2tag(hidden_layer[0].view(batch_size, -1))
        y = F.log_softmax(y, dim=1)
        return y

In [73]:
# label_list = "YN WH please plain".split()
# label_dict = dict( zip(label_list, range(len(label_list))) )

EMBEDDING_DIM = 300
HIDDEN_DIM = EMBEDDING_DIM*2
OUTPUT_DIM = len(remain_classes)


In [74]:
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, BATCH_SIZE)
if torch.cuda.is_available():
   model.cuda()
loss_function = nn.NLLLoss()
# loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)

In [75]:
losses = []
loss_border = 0.0001
# print("error[{0}]".format(error_types[error_i]))
for epoch in range(epoch_):  # again, normally you would NOT do 300 epochs, it is toy data
    all_loss = 0
    for data in trainloader:
        X_t_tensor = torch.tensor(data[0], device='cuda:0')
        # print(X_t_tensor.shape , y_t_tensor.view(-1,1).shape)
        # y_t_tensor = torch.tensor(data[1].reshape(batch_size, 1), device='cuda:0').float()
        y_t_tensor = torch.tensor(data[1], device='cuda:0')
        optimizer.zero_grad()
        model.zero_grad()
        # print(X_t_tensor.shape)

        score = model(X_t_tensor)
        # print(X_t_tensor.shape, score.view(-1,5).shape, y_t_tensor.view(-1,1).shape)
        loss_ = loss_function(score,  y_t_tensor)
        loss_.backward()
        all_loss += loss_.item()
        optimizer.step()
        del score
        del loss_
    losses.append(all_loss)
    if (epoch+1) % 50 == 0:
        print("epoch", epoch+1, "\t" , "loss", all_loss)
    # if all_loss <= loss_border:
    #     print("loss was under border(={0}) : train end".format(loss_border))
    #     break
print("done")

/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


epoch 50 	 loss 87.31120872497559
epoch 100 	 loss 87.33380079269409
epoch 150 	 loss 87.32904863357544
epoch 200 	 loss 87.32049083709717
epoch 250 	 loss 87.3612232208252
epoch 300 	 loss 87.31687474250793
epoch 350 	 loss 87.3333375453949
epoch 400 	 loss 87.3153965473175
epoch 450 	 loss 87.34970760345459
epoch 500 	 loss 87.31825542449951
epoch 550 	 loss 87.3472146987915
epoch 600 	 loss 87.3267834186554


KeyboardInterrupt: 

In [ ]:
X_test = make_Xseq(X_test_str)

In [ ]:
with torch.no_grad():
    X_tensor = torch.tensor(X_test, device='cuda:0').float()
    # y_tensor = torch.tensor(y_test, dtype=torch.long, device='cuda:0')
            # 推論
    y_pred= np.array(model(X_tensor).cpu()).argmax(axis=1)

/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.2682926829268293